In [1]:
import os
import time
from transformers import AutoFeatureExtractor, AutoModelForImageClassification, ViTImageProcessor
import torch
from PIL import Image
from sklearn.metrics import accuracy_score
import torch_tensorrt
import numpy

In [2]:
# Путь к нашему ViT, который будем конвертировать.
path_to_model = "weights/my_model"

extractor = ViTImageProcessor.from_pretrained(path_to_model)
vit_model = AutoModelForImageClassification.from_pretrained(path_to_model)

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpb4nxsmuk
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpb4nxsmuk/_remote_module_non_scriptable.py


In [3]:
def model_use(model, img):
    with torch.no_grad():
        logits = model(**img).logits

    predicted_label = logits.argmax(-1).item()

    return model.config.id2label[predicted_label]


# Функция для запуска конвертированной в ONNX модели. 
def trt_model_use(model, img):
    
    model_id2label=  {0: "cats", 1: "dogs"}
    output = model(img)

    predicted_label = output.logits[0].argmax(-1).item()

    return model_id2label[predicted_label]

In [4]:
# Путь к тестовым картинкам.
path_to_images = "data/"

images_list = os.listdir(path_to_images)

In [5]:
# Функция для замера размера модели.
def size_measurement(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / (1024 ** 2)
    print('model size: {:.3f}MB'.format(size_all_mb))

In [6]:
# Найдем исходный размер модели.
size_measurement(vit_model)

model size: 327.302MB


In [7]:
model = vit_model.eval().to("cuda")

image = Image.open(path_to_images + images_list[1], mode='r', formats=None)
inputs = extractor(image, return_tensors="pt").to("cuda")

# Enabled precision for TensorRT optimization
enabled_precisions = {torch.float}

# Whether to print verbose logs
debug = True

# Workspace size for TensorRT
workspace_size = 20 << 30

# Maximum number of TRT Engines
# (Lower value allows more graph segmentation)
min_block_size = 7

# Operations to Run in Torch, regardless of converter support
torch_executed_ops = {}

# Define backend compilation keyword arguments
compilation_kwargs = {
    "enabled_precisions": enabled_precisions,
    "debug": debug,
    "workspace_size": workspace_size,
    "min_block_size": min_block_size,
    "torch_executed_ops": torch_executed_ops,
}

# Build and compile the model with torch.compile, using Torch-TensorRT backend
optimized_model = torch.compile(
    model,
    backend="torch_tensorrt",
    options=compilation_kwargs,
)

output = optimized_model(inputs['pixel_values'])

[2023-10-14 16:02:30,578] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing forward
[2023-10-14 16:02:31,682] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo done tracing forward (RETURN_VALUE)
[2023-10-14 16:02:31,704] torch._dynamo.output_graph: [INFO] Step 2: calling compiler function torch_tensorrt_backend
INFO:torch_tensorrt.fx.fx2trt:TRT INetwork construction elapsed time: 0:00:00.002091


[10/14/2023-16:02:45] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO:torch_tensorrt.fx.fx2trt:Build TRT engine elapsed time: 0:00:04.373623
INFO:torch_tensorrt.fx.fx2trt:TRT INetwork construction elapsed time: 0:00:00.001781


[10/14/2023-16:02:49] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:02:49] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO:torch_tensorrt.fx.fx2trt:Build TRT engine elapsed time: 0:00:04.038365
INFO:torch_tensorrt.fx.fx2trt:TRT INetwork construction elapsed time: 0:00:00.002213


...

[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up 

INFO:torch_tensorrt.fx.fx2trt:Build TRT engine elapsed time: 0:00:00.113709
INFO:torch_tensorrt.fx.fx2trt:TRT INetwork construction elapsed time: 0:00:00.001502
INFO:torch_tensorrt.fx.fx2trt:Build TRT engine elapsed time: 0:00:00.112205
INFO:torch_tensorrt.fx.fx2trt:TRT INetwork construction elapsed time: 0:00:00.002714


[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[10/14/2023-16:03:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up 

INFO:torch_tensorrt.fx.fx2trt:Build TRT engine elapsed time: 0:00:00.320683
[2023-10-14 16:03:19,183] torch._dynamo.output_graph: [INFO] Step 2: done compiler function torch_tensorrt_backend


[10/14/2023-16:03:19] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [8]:
# Проверим размер trt
size_measurement(optimized_model)

model size: 327.302MB


In [9]:
# Запустим тест нашей trt модели.

start_time = time.time()

# Собака 1, кошка 0.
target_list = []
predict_list = []

for element in images_list:

    image = Image.open(path_to_images + element, mode='r', formats=None)

    inputs = extractor(image, return_tensors="pt")['pixel_values']
    predict = trt_model_use(optimized_model, inputs.to("cuda"))
    target = element[:element.find(".")]

    if target == "dog":
        label = 1
    else:
        label = 0

    target_list.append(label)

    if predict == "dogs":
        pr = 1
    else:
        pr = 0

    predict_list.append(pr)

end_time = time.time()

acc = accuracy_score(target_list, predict_list)
print("Точность сконвертированной в TRT модели= ", acc)
print("Время обработки изображений сконвертированной в TRT моделью = ", end_time-start_time, " секунд")
print("Скорость обработки изображений у сконвертированной в TRT модели составила  ", len(images_list)/(end_time-start_time), " картинок в секунду")

Точность сконвертированной в TRT модели=  0.9875
Время обработки изображений сконвертированной в TRT моделью =  6.732074975967407  секунд
Скорость обработки изображений у сконвертированной в TRT модели составила   23.76681789361798  картинок в секунду
